# Predictive Modeling of County-Level Acute Food Insecurity in Kenya

## Business Understanding

### Context and Motivation

Food insecurity remains a persistent challenge in Kenya, particularly in arid and semi-arid lands (ASALs). According to the [World Food Programme – Kenya](https://www.wfp.org/countries/kenya), **approximately 4.3 million people were projected to face acute food insecurity during the peak of the 2023 drought**, highlighting the scale of climate-driven vulnerability. The [Global Hunger Index – Kenya](https://www.globalhungerindex.org/kenya.html) classifies Kenya’s hunger level as **“serious”**, indicating ongoing structural food security challenges. Projections from the [EU Knowledge for Policy platform](https://knowledge4policy.ec.europa.eu/publication/kenya-acute-food-insecurity-situation-july-september-2024-projection-october-2024_en) further showed that **multiple counties were expected to experience IPC Phase 3 (Crisis) or worse during seasonal periods in 2024**.

These recent assessments demonstrate that large populations in Kenya repeatedly face acute food insecurity. However, current tools such as IPC primarily describe conditions after they occur. This limits early action and often leads to reactive humanitarian response. There is therefore a clear need for predictive systems that can anticipate food insecurity and support proactive decision-making.

---

### Project Goal

This project develops a predictive early-warning model for acute food insecurity risk at the county level in Kenya by integrating historical IPC classifications with key drivers including climate conditions, staple food price dynamics, conflict events, and baseline poverty indicators. The resulting model and dashboard will provide forward-looking risk scores to support early action by county governments, humanitarian organizations, and policy planners, helping shift decision-making from reactive response to anticipatory planning.

---

### Project Objectives

1. Build a county-level early-warning system that anticipates IPC Phase 3+ food insecurity 1–3 months ahead.
2. Reveal the most influential climate, market, conflict, and poverty drivers shaping food insecurity risk across Kenya.
3. Produce reliable risk scores and hotspot signals to support timely policy and humanitarian action.
4. Deliver a decision-ready dashboard that visualizes risk trends, spatial patterns, and key contributing factors.

---

## Data Understanding

This project leverages five key data sources that represent both the target outcome and predictive signals:

### IPC Kenya Classifications

The IPC dataset provides historical classifications of food insecurity severity (Phase 1–5) by county and period. It serves as the **target variable** for supervised learning by flagging whether a county is in crisis (Phase 3 or worse) at a given time.

### Subnational Rainfall Indicators

Rainfall data contains monthly totals, historical averages, and anomalies for Kenyan counties. These indicators capture **climate stress**, particularly drought conditions, which are a known driver of crop failure and livestock losses.

### WFP Food Prices

Food price data tracks the cost of staple commodities — such as maize, beans, and rice — in various markets over time. Changes in staple food prices reflect **market stress** and reduced food access for net buyers, especially among low-income households.

### Political Violence Events

Conflict event data records the number of violent events and associated fatalities by county and month. Political violence disrupts markets, displacement, and access to humanitarian support, making it a critical **shock indicator** for food insecurity modeling.

### Multidimensional Poverty Index (MPI)

The MPI dataset provides county-level socioeconomic vulnerability measures, including headcount poverty rates and intensity of deprivation. These indicators act as **baseline contextual features**, helping the model account for structural vulnerability that can exacerbate the impact of shocks.

### 1. Notebook Setup & Environment

In [1]:
# Core data handling
import pandas as pd
import numpy as np

# Date and time utilities
from datetime import datetime

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Gradient Boosting
import xgboost as xgb

# Imbalance handling
from imblearn.over_sampling import SMOTE

# Model explainability
import shap

# File system utilities
from pathlib import Path
import os

# Notebook display settings
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:.4f}".format)

# Visualization style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Project directory structure
BASE_DIR = Path.cwd()

DATA_DIR = BASE_DIR / "data"
MODELS_DIR = BASE_DIR / "models"
OUTPUTS_DIR = BASE_DIR / "outputs"
FIGURES_DIR = OUTPUTS_DIR / "figures"

# Create directories if they don't exist
for directory in [DATA_DIR, MODELS_DIR, OUTPUTS_DIR, FIGURES_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print("Project directories initialized.")
print(f"Base directory: {BASE_DIR}")

Project directories initialized.
Base directory: c:\Users\Jasho\Documents\Moringa\Phase 5\Food Insecurity\PHASE-5-PROJECT---PREDICTING-FOOD-INSECURITY\notebooks


### 2. Data Loading & Inventory

In [4]:
# 2.1 Define File Paths
BASE_DIR = Path.cwd().parent if Path.cwd().name.lower() == "notebooks" else Path.cwd()
DATA_DIR = BASE_DIR / "data"

ipc_path = DATA_DIR / "ipc_ken_area_long.csv"
rainfall_path = DATA_DIR / "ken-rainfall-subnat-full.csv"
prices_path = DATA_DIR / "wfp_food_prices_ken.csv"
conflict_path = DATA_DIR / "kenya_political_violence_events_and_fatalities_by_month-year_as-of-11feb2026.xlsx"
mpi_path = DATA_DIR / "ken_mpi.csv"

# Quick existence check for all files
paths = {
    "IPC": ipc_path,
    "Rainfall": rainfall_path,
    "Food Prices": prices_path,
    "Conflict": conflict_path,
    "MPI": mpi_path,
}
missing_files = [name for name, p in paths.items() if not p.exists()]
if missing_files:
    print("Base directory:", BASE_DIR)
    print("Data directory:", DATA_DIR)
    for name, p in paths.items():
        print(f"{name} -> {p} (exists={p.exists()})")
    raise FileNotFoundError(f"Missing file(s) in DATA_DIR: {missing_files}")

# 2.2 Load Raw Datasets
def _read_csv_safely(path: Path) -> pd.DataFrame:
    """Read CSV with a safe fallback for encoding."""
    try:
        return pd.read_csv(path)
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="latin-1")

ipc_df = _read_csv_safely(ipc_path)
rainfall_df = _read_csv_safely(rainfall_path)
prices_df = _read_csv_safely(prices_path)
conflict_df = pd.read_excel(conflict_path, sheet_name="Data")
mpi_df = _read_csv_safely(mpi_path)

# 2.3 Initial Inspection
datasets = {
    "IPC": ipc_df,
    "Rainfall": rainfall_df,
    "Food Prices": prices_df,
    "Conflict": conflict_df,
    "MPI": mpi_df
}

for name, df in datasets.items():
    print(f"\n{name} Dataset")
    print("-" * 40)
    print("Shape:", df.shape)
    print("\nColumns:")
    print(df.columns.tolist())
    print("\nPreview:")
    display(df.head())

# 2.4 Data Types Check
for name, df in datasets.items():
    print(f"\n{name} Data Types")
    print("-" * 40)
    display(df.dtypes)

# 2.5 Missing Value Summary
for name, df in datasets.items():
    print(f"\n{name} Missing Values")
    print("-" * 40)
    display(df.isna().sum().sort_values(ascending=False))


IPC Dataset
----------------------------------------
Shape: (4522, 11)

Columns:
['Date of analysis', 'Country', 'Total country population', 'Level 1', 'Area', 'Validity period', 'From', 'To', 'Phase', 'Number', 'Percentage']

Preview:


,Date of analysis,Country,Total country population,Level 1,Area,Validity period,From,To,Phase,Number,Percentage
0,Jul 2025,KEN,16617000,Others,Marsabit,current,2025-07-01,2025-09-30,all,515000,1.0000
1,Jul 2025,KEN,16617000,Others,Marsabit,current,2025-07-01,2025-09-30,3+,103000,0.2000
2,Jul 2025,KEN,16617000,Others,Marsabit,current,2025-07-01,2025-09-30,1,128750,0.2500
3,Jul 2025,KEN,16617000,Others,Marsabit,current,2025-07-01,2025-09-30,2,283250,0.5500
4,Jul 2025,KEN,16617000,Others,Marsabit,current,2025-07-01,2025-09-30,3,103000,0.2000



Rainfall Dataset
----------------------------------------
Shape: (131544, 15)

Columns:
['date', 'adm_level', 'adm_id', 'PCODE', 'n_pixels', 'rfh', 'rfh_avg', 'r1h', 'r1h_avg', 'r3h', 'r3h_avg', 'rfq', 'r1q', 'r3q', 'version']

Preview:


,date,adm_level,adm_id,PCODE,n_pixels,rfh,rfh_avg,r1h,r1h_avg,r3h,r3h_avg,rfq,r1q,r3q,version
0,1981-01-01,1,51325,KE019,427.0000,7.3724,15.7594,NaN,NaN,NaN,NaN,59.5988,NaN,NaN,final
1,1981-01-11,1,51325,KE019,427.0000,4.3255,19.2948,NaN,NaN,NaN,NaN,38.3849,NaN,NaN,final
2,1981-01-21,1,51325,KE019,427.0000,5.5691,16.2654,17.2670,51.3196,NaN,NaN,49.7008,39.5368,NaN,final
3,1981-02-01,1,51325,KE019,427.0000,5.8829,12.7193,15.7775,48.2795,NaN,NaN,61.4184,38.9972,NaN,final
4,1981-02-11,1,51325,KE019,427.0000,17.1803,18.7686,28.6323,47.7533,NaN,NaN,93.3177,63.7539,NaN,final



Food Prices Dataset
----------------------------------------
Shape: (17365, 16)

Columns:
['date', 'admin1', 'admin2', 'market', 'market_id', 'latitude', 'longitude', 'category', 'commodity', 'commodity_id', 'unit', 'priceflag', 'pricetype', 'currency', 'price', 'usdprice']

Preview:


...

admin1          50
admin2          50
longitude       50
latitude        50
market           0
date             0
market_id        0
category         0
commodity        0
commodity_id     0
unit             0
priceflag        0
pricetype        0
currency         0
price            0
usdprice         0
dtype: int64


Conflict Missing Values
----------------------------------------


Country       0
Month         0
Year          0
Events        0
Fatalities    0
dtype: int64


MPI Missing Values
----------------------------------------


Admin 1 PCode               1
Admin 1 Name                1
Country ISO3                0
MPI                         0
Headcount Ratio             0
Intensity of Deprivation    0
Vulnerable to Poverty       0
In Severe Poverty           0
Survey                      0
Start Date                  0
End Date                    0
dtype: int64